### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import random
import math
from collections import deque
import collections
import pickle
import os
import time


# for building DQN model 
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver

#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

In [3]:
Time_matrix.shape

(5, 5, 24, 7)

In [4]:
Time_matrix[1][2][11][3]

10.0

In [5]:
Time_matrix.max()

11.0

Maximum time to travel between two locations is 11 hours (next state can change by max of 1 day)

#### Tracking the state-action pairs for checking convergence


In [6]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### TODO  : Picking the architecture

#### Architecture 1: input - state, output - q-value for each possible (state, action)

Pros:
- Model has to be run once for each state  

Cons:
    

#### Architecture 2: input - (state, action), output - q-value for the given (state, action)

#### Conclusion

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN 
        self.discount_factor = 0.99
        self.learning_rate = 0.001       
        self.epsilon_max = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.00000001
        self.epsilon = 1
    
        self.explore_count = 0
        self.exploit_count = 0
        
        self.batch_size = 32        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # Initialize the value of the states tracked
        self.states_tracked = []
        
        # Track the state [1,0,0] and action (1,3) at index 7 in the action space.
        self.track_state = np.array(env.state_encod_arch1([1,0,0])).reshape(1, 36)
        
        # create main model and target model
        self.model = self.build_model()
        
        
        
        

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        
        # input layer
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform')) 
        
        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform')) 
        
        
        
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model

     

    def get_action(self, state, episode_count):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment       
        
        
#         exp_value = 1/10**(len(str(episode_count))-1)
#         self.epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(-exp_value*episode_count)
    
        
        
        if np.random.rand() <= self.epsilon:
            # exploration: choose a random action from all possible actions
            ##print(f"{episode_count}: EXPLORE")
            self.explore_count += 1
            return random.randrange(self.action_size)

        else:
            # exploitation: choose the action that returns the maximum q-value
            ##print(f"{episode_count}: EXPLOIT")
            
            
#             # the first index corresponds to the batch size, so
#             # reshape state to (1, state_size) so that the first index corresponds to the batch size 
            state = np.array(env.state_encod_arch1(state)).reshape(1, 36)
            #state = np.reshape(state,[1,self.state_size])
            q_value = self.model.predict(state) 
            self.exploit_count += 1
            return np.argmax(q_value)
         
 
   
    def get_summary_details(self):
        return self.explore_count, self.exploit_count
        

        
    def reset_episode_counts(self):
        self.explore_count = 0
        self.exploit_count = 0

    def append_sample(self, state, action_idx, reward, next_state, done):
    # Write your code here:
    # save sample <s,a,r,s',done> to the replay memory after every action
        self.memory.append((state, action_idx, reward, next_state, done))
        
        
#         # update ε after each sample
#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay    
    
    

    # Pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        """
        train the neural network on a minibatch. Input to the network is the states,
        output is the target q-value corresponding to each action.
        """

        if len(self.memory) > self.batch_size:
            
            # sample minibatch from memory
            minibatch = random.sample(self.memory, self.batch_size)

            # initialise two matrices - update_input and update_output
            # Initialise the Q(s,a) with zero
            update_input = np.zeros((self.batch_size, self.state_size))
            # Initialise the Q(s',a)
            update_output = np.zeros((self.batch_size, self.state_size))
            actions, rewards, done = [], [], []

            # populate update_input and update_output and the lists rewards, actions, done
            for i in range(self.batch_size):
                state, action_idx, reward, next_state, is_done = minibatch[i]
                #print(f'4DEBUG:minibatch[{i}] = {minibatch[i]}')
                # Add state s to the Q(s,a), Q(s',a) from memory
                #print(f'4DEBUG: state:{state}, encoded state: {env.state_encod_arch1(state)}')
                update_input[i] = env.state_encod_arch1(state)     
                #update_input[i] = state
                # Add action from memory
                actions.append(action_idx)
                # Add reward from the memory
                rewards.append(reward)
                # Add next state s' to Q(s',a) from the memory
                update_output[i] = env.state_encod_arch1(next_state)
                #update_output[i] = next_state
                done.append(is_done)

            # Find the Q(s,a) and Q(s',a) using state as input to the neural network 
        
            # Predict the target q-values from state s
            target = self.model.predict(update_input)

            # Get the target for the Q-network
            target_qval = self.model.predict(update_output)
            
            #print(f'4DEBUG: target:{target}')
            #print(f'4DEBUG: target_qval:{target_qval}')

            # Update the target values - set the target as (r + maxQ(s',a))
            for i in range(self.batch_size):
                # Q Learning: get maximum Q value at s' from target model
                if done[i]:
                    target[i][actions[i]] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])

            # Train the model
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
     

    def save_tracking_states(self):
        # Use the model to predict the q_value for the state tracked
        q_value = self.model.predict(self.track_state)
        print(f'4DEBUG:save_tracking_states - {q_value}')
        # Update the q-value for the state tracked
        self.states_tracked.append(q_value[0][7])
        
    def save_model_weights(self, name):
        self.model.save_weights(name)

    def save(self, name):
        self.model.save(name)

In [8]:
Episodes = 10000

### DQN block

In [9]:

env = CabDriver()

m = 5
t = 24
d = 7
# get size of state and action from environment
state_size = m+t+d
action_size = len(env.action_space)

print(f'State size: {state_size}, Action size: {action_size}')

# agent needs to be initialised outside the loop since the DQN
# network will be initialised along with the agent
agent = DQNAgent(action_size=action_size, state_size=state_size) 

# to store rewards in each episode
rewards_per_episode, episodes = [], []

# make dir to store model weights
if not os.path.exists("saved_model_weights"):
    os.mkdir("saved_model_weights")
 

summary_df = pd.DataFrame(columns=['EPISODE','REWARD','MEMORY_LENGTH','EPSILON','EXPLORE_CNT', 'EXPLOIT_CNT', 'EPISODE_TIME'])

summary_threshold = 1000
def update_summary_details(episode,reward, memory_len, epsilon, explore_count, exploit_count,episode_time):
    #print(f'Updating summary details:{match_results}')
    return summary_df.append({'EPISODE' : episode,
                       'REWARD' : reward,
                       'MEMORY_LENGTH' : memory_len,
                       'EPSILON' : epsilon,
                       'EXPLORE_CNT' : explore_count,
                       'EXPLOIT_CNT' : exploit_count,
                       'EPISODE_TIME': episode_time
                      }, ignore_index=True) 

State size: 36, Action size: 21





for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    

    #Call the DQN agent
    
    
    while !terminal_state:
        
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        # 2. Evaluate your reward and next state
        # 3. Append the experience to the memory
        # 4. Train the model by calling function agent.train_model
        # 5. Keep a track of rewards, Q-values, loss
        

In [ ]:
start_time = time.time()

#### simulation starts ####
for episode in range(Episodes):
    episode_start_time = time.time()
    done = False
    score = 0

    # reset at the start of each episode
    state = env.reset()

    while not done:

        # 1. Pick epsilon-greedy action from possible actions for the current state
        action_idx = agent.get_action(state,episode)
        action = env.action_space[action_idx]
        # 2. Evaluate your reward and next state
        next_state, reward, done = env.step(state, action, Time_matrix)
            
        #next_state, reward, done, info = env.step(action)

        # 3. Append the experience to the memory
        # save the sample <s, a, r, s', done> to the replay memory
        agent.append_sample(state, action_idx, reward, next_state, done)

        # 4. Train the model by calling function agent.train_model
        agent.train_model()

        # add reward to the total score of this episode
        score += reward
        state = next_state



    # epsilon decay
    if agent.epsilon > agent.epsilon_min:
        agent.epsilon *= agent.epsilon_decay
#         agent.epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(agent.epsilon_decay*episode)

    # 5. Keep a track of rewards, Q-values, loss
    # store total reward obtained in this episode
    rewards_per_episode.append(score)
    episodes.append(episode)
    
    # Save summary after every episode
    explore_count, exploit_count = agent.get_summary_details()
    episode_time = time.time() - episode_start_time
    summary_df = update_summary_details(episode,score, len(agent.memory),round(agent.epsilon,5),explore_count, exploit_count,round(episode_time,3))
    agent.reset_episode_counts()
        
    # every episode:
    print(f"episode {episode}, reward {score}, memory_length {len(agent.memory)}, epsilon {round(agent.epsilon,5)}, explore:{explore_count}, exploit:{exploit_count},time: {round(episode_time,3)}")
        
    # Save the Q_value of the state, action pair we are tracking
    if ((episode + 1) % 5 == 0):
        agent.save_tracking_states()
        
        
    # every few episodes:
    if episode % 100 == 0:
        # store q-values of some prespecified state-action pairs
        # q_dict = agent.store_q_values()
        
        
        curr_elapsed_time = time.time() - start_time
        print(f'Episodes:{episode} - Elapsed_time:{round(curr_elapsed_time,3)} ')  

        # save model weights
        agent.save_model_weights(name="model_weights.h5")

 
# save model weights
agent.save_model_weights(name="model_weights.h5")
    
elapsed_time = time.time() - start_time
print(f'Total elapsed_time:{elapsed_time}')       
    
        
#### simulation complete ####

episode 0, reward 103.0, memory_length 122, epsilon 0.999, explore:122, exploit:0,time: 7.875
Episodes:0 - Elapsed_time:7.877 
episode 1, reward -95.0, memory_length 242, epsilon 0.998, explore:120, exploit:0,time: 8.415
episode 2, reward 67.0, memory_length 364, epsilon 0.997, explore:122, exploit:0,time: 8.46
episode 3, reward -65.0, memory_length 479, epsilon 0.99601, explore:113, exploit:2,time: 8.209
episode 4, reward -169.0, memory_length 593, epsilon 0.99501, explore:114, exploit:0,time: 8.117
4DEBUG:save_tracking_states - [[    0.    43993.43  42998.03  21654.29  41827.25  42556.1   43334.363
      0.    42643.934     0.        0.    42910.95      0.    41047.617
  43267.586 42514.113     0.        0.    43234.45  42794.117 43559.688]]
episode 5, reward 9.0, memory_length 719, epsilon 0.99401, explore:125, exploit:1,time: 8.733
episode 6, reward 107.0, memory_length 840, epsilon 0.99302, explore:120, exploit:1,time: 8.566
episode 7, reward -118.0, memory_length 951, epsilon 0.9

In [ ]:

plt.figure(0, figsize=(16,8))
plt.plot(summary_df[['EPISODE','REWARD']])
plt.title('REWARDS PER EPISODE')
plt.xlabel("Episode")
plt.ylabel("Reward")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards_episodes.png')

In [ ]:
display(summary_df)

In [ ]:
# save stuff as pickle
def save_pickle(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# make directory
if not os.path.exists("saved_pickle_files"):
    os.mkdir("saved_pickle_files")

# save rewards_per_episode
save_pickle(rewards_per_episode, "saved_pickle_files/rewards_per_episode")
save_pickle(summary_df, "saved_pickle_files/summary_per_episode")



### Tracking Convergence

#### Tracked state

In [ ]:
agent.states_tracked

In [ ]:
state_tracked_sample = [agent.states_tracked[i] for i in range(len(agent.states_tracked)) if agent.states_tracked[i] < 1000]

#### Q-value convergence plot for the tracked state

In [ ]:
plt.figure(0, figsize=(16,7))
plt.title('Q_value for state [1,0,0]  action (1,3)')
xaxis = np.asarray(range(0, len(agent.states_tracked)))
plt.semilogy(xaxis,np.asarray(agent.states_tracked))
plt.show()

#### Plot Episode vs Reward

In [ ]:

# plot results
with open('saved_pickle_files/summary_per_episode.pkl', 'rb') as f:
    summary_data = pickle.load(f)

    
plt.figure(0, figsize=(16,8))
plt.plot(summary_data[['EPISODE','REWARD']])
plt.title('REWARDS PER EPISODE')
plt.xlabel("Episode")
plt.ylabel("Reward")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards_episodes.png')
  

In [ ]:

# plot results
with open('saved_pickle_files/rewards_per_episode.pkl', 'rb') as f:
    rewards_per_episode = pickle.load(f)

plt.plot(list(range(len(rewards_per_episode))), rewards_per_episode)
plt.xlabel("episode number")
plt.ylabel("reward per episode")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards.png')


print("Average reward of last 100 episodes is {0}".format(np.mean(rewards_per_episode[-100:]))) 









#### Plot exploration vs exploitation

In [ ]:

plt.figure(0, figsize=(16,8))
plt.plot(summary_data[['EXPLORE_CNT','EXPLOIT_CNT']])
plt.title('EXPLORATION vs EXPLOITATION')
plt.xlabel("Exploration")
plt.ylabel("Exploitation")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('exploration_exploitation.png')

### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
import matplotlib.pyplot as plt

max_epsilon = 1.0
min_epsilon = 0.0000001
episode_count = 25000
exp_value = 1/10**(len(str(episode_count))-1)
print(f'exp:{exp_value}')
time = np.arange(0,episode_count)
epsilon = []
for i in range(0,episode_count):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-exp_value*i))
    

plt.plot(time, epsilon)
plt.show()

In [ ]:
# From starter code
total_episodes = 10000
time = np.arange(0,total_episodes)
epsilon = []
for i in range(0,total_episodes):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))
    
plt.plot(time, epsilon)
plt.show()

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
episode_count = 50
exp_value = 1/10**(len(str(episode_count))-1)
time = np.arange(0,episode_count)
epsilon = []
for i in range(0,episode_count):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-exp_value*i))
    

plt.plot(time, epsilon)
plt.show()